In [2]:
import requests
import base64
import datetime
from urllib.parse import urlencode
# from secrets import spotify_user_id

In [3]:
spotify_user_id = "ed.doris"
client_id = "6f2a7816998d4d7e992f1dba605ccd33"
client_secret = "f1029bcc83634b4c9497ec0bcd701498"
a={}

In [39]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    playlist_name = "meta_playlist"
    
    token_url ="https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
        
    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
        
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
        "Authorization": f"Basic {client_creds_b64}"
        }
    
    def get_token_data(self):
                return {
            "grant_type": "client_credentials"
        }
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200, 299):
            raise Exception("Could not authenticate client.")   
#             return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token()
        return token
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
        "Authorization": f"Bearer {access_token}"
        }
        return headers
    
    def get_resource(self, lookup_id, resource_type="albums", version="v1"):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
    def get_album(self, _id):
        return self.get_resource(_id, resource_type = "albums")
    
    def get_artist(self, _id):
        return self.get_resource(_id, resource_type = "artists")

    
    def base_search(self, query_params):
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"

        lookup_url = f"{endpoint}?{query_params}"
        print(lookup_url)
        r = requests.get(lookup_url, headers=headers)
        print(r.status_code)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
    def search(self, query=None, search_type="albums"):
        if query == None:
            raise Exception("Need query_params for search")
        if isinstance(query, dict):
            query = " ".join([f"{k}:{v}" for k,v in query.items()])
        query_params = urlencode({"q":query, "type":search_type.lower()})
        print(query_params)
        return self.base_search(query_params)


In [34]:
def create_playlist(self, name):
    endpoint = "https://api.spotify.com/v1/users/"
    headers = self.get_resource_header()
    create_playlist_params = urlencode({"Name": name})
    create_playist_url = f"{endpoint}?{create_playlist_params}"
    r = requests.post(create_playist_url, headers=headers)
    print(r)

In [40]:
playlist_name.create_playlist()

NameError: name 'playlist_name' is not defined

In [4]:
spotify = SpotifyAPI(client_id, client_secret)

In [5]:
a = spotify.search({"album":"Fetch the Bolt Cutters", "artist":"Fiona Apple"}, search_type="album")

q=album%3AFetch+the+Bolt+Cutters+artist%3AFiona+Apple&type=album
https://api.spotify.com/v1/search?q=album%3AFetch+the+Bolt+Cutters+artist%3AFiona+Apple&type=album
200


In [6]:
print(a)

{'albums': {'href': 'https://api.spotify.com/v1/search?query=album%3AFetch+the+Bolt+Cutters+artist%3AFiona+Apple&type=album&offset=0&limit=20', 'items': [{'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3g2kUQ6tHLLbmkV7T4GPtL'}, 'href': 'https://api.spotify.com/v1/artists/3g2kUQ6tHLLbmkV7T4GPtL', 'id': '3g2kUQ6tHLLbmkV7T4GPtL', 'name': 'Fiona Apple', 'type': 'artist', 'uri': 'spotify:artist:3g2kUQ6tHLLbmkV7T4GPtL'}], 'available_markets': ['AD', 'AE', 'AL', 'AR', 'AT', 'AU', 'BA', 'BE', 'BG', 'BH', 'BO', 'BR', 'BY', 'CA', 'CH', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DE', 'DK', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FR', 'GB', 'GR', 'GT', 'HK', 'HN', 'HR', 'HU', 'ID', 'IE', 'IL', 'IN', 'IS', 'IT', 'JO', 'JP', 'KW', 'KZ', 'LB', 'LI', 'LT', 'LU', 'LV', 'MA', 'MC', 'MD', 'ME', 'MK', 'MT', 'MX', 'MY', 'NI', 'NL', 'NO', 'NZ', 'OM', 'PA', 'PE', 'PH', 'PL', 'PS', 'PT', 'PY', 'QA', 'RO', 'RS', 'RU', 'SA', 'SE', 'SG', 'SI', 'SK', 'SV', 'TH', 'TN', 'TR